<a href="https://colab.research.google.com/github/has-c/UserMobilityMining/blob/master/Gowalla%20ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import keras
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Embedding, Softmax
from sklearn.metrics import accuracy_score
import datetime

#disable warnings
import warnings
warnings.filterwarnings("ignore")

Using TensorFlow backend.
/home/usermobilitymining/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/usermobilitymining/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/usermobilitymining/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/usermobilitymining/.loc

## Helper Functions

In [47]:
def alignPastCurrentPOI(checkins, dateList):

    #group by userId, then checkin date and then order by time 
    uniqueUserIds = np.unique(checkins['userid'])
    uniqueCheckinDates = sorted(np.unique(dateList))

    sortedCheckins = pd.DataFrame([])

    for userId in uniqueUserIds:
        for checkinDate in uniqueCheckinDates:
            
            #group by userId and checkin date 
            groupedByUserId = checkins[userId==checkins['userid']]
            groupedByCheckinDate = groupedByUserId[groupedByUserId['Date'] == checkinDate]
            if groupedByCheckinDate.shape[0] == 0:
                continue
                
            #sort by time 
            groupedByCheckinDate.sort_values(by ='Time',ascending=True, inplace=True)
            groupedByCheckinDate.index = range(0, groupedByCheckinDate.shape[0])
            
            #create list of sorted checkins 
            sortedCheckins = pd.concat([sortedCheckins, groupedByCheckinDate], axis=0, ignore_index=True)
    
    #align past and current checkins - features and labels entire df
    pastCurrentCheckins = pd.DataFrame([])

    for userId in uniqueUserIds:
        
        #group by userId
        groupedByUserId = sortedCheckins[userId==checkins['userid']]
        
        for rowIndex in groupedByUserId.index:
            try:
                pastPOI = groupedByUserId.loc[rowIndex]
                currentPOI = groupedByUserId.loc[rowIndex+1]
                poiInstance = list(currentPOI) + list(pastPOI)
                pastCurrentCheckins = pd.concat([pastCurrentCheckins, pd.DataFrame(poiInstance).T], axis=0)
            except KeyError:
                break

    #rename fields 
    pastCurrentCheckinDfColumnNames = ['Current ' + name for name in sortedCheckins.columns] + ['Past ' + name for name in sortedCheckins.columns]
    pastCurrentCheckins.columns = pastCurrentCheckinDfColumnNames

    return pastCurrentCheckins

### Preprocessing 
<ul>
    <li> One hot encoding </li>
    <li> Standard scaling </li>
</ul>

In [4]:
#import data
checkinsDf = pd.read_csv("/home/usermobilitymining/Notebooks/volume/Hasnain/Processed Data/processedCheckins.csv")
nzCheckins = pd.read_csv("/home/usermobilitymining/Notebooks/volume/Hasnain/Processed Data/nzCheckinsWithGridTokens.csv")

In [6]:
#join checkins df and nz checkins to only extract NZ checkins 
checkinsDf['id'] = checkinsDf.index #create id field
nzCheckins.drop("lat", axis=1, inplace=True) #drop lat so no error
nzCheckins.drop("lng", axis=1, inplace=True) #drop lng so no error
checkinsDf = nzCheckins.join(checkinsDf.set_index('id'), on='id')
checkinsDf.drop("Unnamed: 0", axis=1, inplace=True)

In [13]:
#convert date and time to datetime variables
#extract day of the week and hour of checkin
checkinDate = [datetime.datetime.strptime(date, "%Y-%m-%d").date() for date in checkinsDf['Date']]
checkinDay = [date.weekday() for date in checkinDate]
checkinTime = [datetime.datetime.strptime(time, '%H:%M:%S').time() for time in checkinsDf['Time']]
checkinHour = [time.hour for time in checkinTime]

In [15]:
#one hot encode the time variable
ohe = OneHotEncoder(dtype=np.int8, n_values=24)
hourOfCheckinEncoded = ohe.fit_transform(np.array(checkinHour).reshape(-1,1))
timeColumns = ['Time ' + str(num) for num in np.arange(0,24)]
timeDf = pd.DataFrame(hourOfCheckinEncoded.toarray(), columns=timeColumns)

#one hot encode the day of the week variable
ohe = OneHotEncoder(dtype=np.int8, n_values=7)
dayOfTheWeekEncoded = ohe.fit_transform(np.array(checkinDay).reshape(-1,1))
dayColumns = ['DayOfWeek ' + str(num) for num in np.arange(0,7)]
dayDf = pd.DataFrame(dayOfTheWeekEncoded.toarray(), columns=dayColumns)

#one hot encode the main category
numberOfMainCategories = len(np.unique(checkinsDf['Main Category']))
labelEncoder = LabelEncoder()
mainCategoriesEncoded = labelEncoder.fit_transform(checkinsDf['Main Category'].values.reshape(-1,1))
ohe = OneHotEncoder(dtype=np.int8, n_values=numberOfMainCategories)
mainCategoriesEncoded = ohe.fit_transform(mainCategoriesEncoded.reshape(-1,1))
categoryDf = pd.DataFrame(mainCategoriesEncoded.toarray(), columns=labelEncoder.classes_)

In [16]:
#concat fields
checkinsDf = pd.concat([checkinsDf, timeDf], axis=1,copy=False) 
checkinsDf = pd.concat([checkinsDf, dayDf], axis=1,copy=False) 
checkinsDf = pd.concat([checkinsDf, categoryDf], axis=1,copy=False) 

In [17]:
#drop fields 
checkinsDf.drop("Date", axis=1, inplace=True)
checkinsDf.drop("Time", axis=1, inplace=True)
checkinsDf.drop("Main Category", axis=1, inplace=True)
checkinsDf.drop("Locations", axis=1, inplace=True)

In [18]:
#train test split - normal 
labelDf = checkinsDf[['Community','Entertainment','Food','Nightlife','Outdoors','Shopping','Travel']]
featuresDf = checkinsDf.drop(['Community','Entertainment','Food','Nightlife','Outdoors','Shopping','Travel'], axis=1)

#train test split
featuresTrain, featuresTest, labelTrain, labelTest = train_test_split(featuresDf, labelDf, test_size=0.2, random_state=42)

### Neural Net

In [34]:
model = Sequential()
model.add(Dense(35, input_dim=45, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(7, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [38]:
#fit model 
history = model.fit(featuresTrain, labelTrain, epochs=100)

Epoch 1/100
556/556 [==============================] - 0s 41us/step - loss: 1675.7278 - accuracy: 0.4083
Epoch 2/100
556/556 [==============================] - 0s 43us/step - loss: 1910.8911 - accuracy: 0.3939
Epoch 3/100
556/556 [==============================] - 0s 38us/step - loss: 1576.3228 - accuracy: 0.4173
Epoch 4/100
556/556 [==============================] - 0s 41us/step - loss: 1354.5961 - accuracy: 0.4299
Epoch 5/100
556/556 [==============================] - 0s 41us/step - loss: 1460.5962 - accuracy: 0.4137
Epoch 6/100
556/556 [==============================] - 0s 41us/step - loss: 1549.8275 - accuracy: 0.4137
Epoch 7/100
556/556 [==============================] - 0s 44us/step - loss: 1379.9547 - accuracy: 0.4173
Epoch 8/100
556/556 [==============================] - 0s 43us/step - loss: 1591.1022 - accuracy: 0.4173
Epoch 9/100
556/556 [==============================] - 0s 41us/step - loss: 1613.1813 - accuracy: 0.4245
Epoch 10/100
556/556 [==============================] -

556/556 [==============================] - 0s 39us/step - loss: 1557.3719 - accuracy: 0.4101
Epoch 79/100
556/556 [==============================] - 0s 48us/step - loss: 1384.4100 - accuracy: 0.4155
Epoch 80/100
556/556 [==============================] - 0s 39us/step - loss: 1712.8653 - accuracy: 0.3705
Epoch 81/100
556/556 [==============================] - 0s 35us/step - loss: 1422.8639 - accuracy: 0.4119
Epoch 82/100
556/556 [==============================] - 0s 40us/step - loss: 1430.6538 - accuracy: 0.4101
Epoch 83/100
556/556 [==============================] - 0s 39us/step - loss: 1401.1207 - accuracy: 0.4406
Epoch 84/100
556/556 [==============================] - 0s 37us/step - loss: 1098.8883 - accuracy: 0.4317
Epoch 85/100
556/556 [==============================] - 0s 37us/step - loss: 1387.8926 - accuracy: 0.3867
Epoch 86/100
556/556 [==============================] - 0s 42us/step - loss: 1124.0422 - accuracy: 0.4227
Epoch 87/100
556/556 [==============================] - 0s 

In [24]:
#predict
labelPredicted = model.predict(featuresTest)